In [1]:
import sys
#!{sys.executable} -m pip install numpy
import pandas as pd
import geopandas as gpd
import numpy as np
from scipy.interpolate import griddata

#!{sys.executable} -m pip install lets-plot

from lets_plot import *
from lets_plot.geo_data import *
LetsPlot.setup_html()

import lets_plot
lets_plot.__version__

income_dat =  pd.read_csv("../Datasets/kaggle_income.csv", encoding="ISO 8859-1")
income_dat.head(3)

The geodata is provided by © OpenStreetMap contributors and is made available here under the Open Database License (ODbL).


,id,State_Code,State_Name,State_ab,County,City,Place,Type,Primary,Zip_Code,Area_Code,ALand,AWater,Lat,Lon,Mean,Median,Stdev,sum_w
0,1011000,1,Alabama,AL,Mobile County,Chickasaw,Chickasaw city,City,place,36611,251,10894952,909156,30.771450,-88.079697,38773,30506,33101,1638.260513
1,1011010,1,Alabama,AL,Barbour County,Louisville,Clio city,City,place,36048,334,26070325,23254,31.708516,-85.611039,37725,19528,43789,258.017685
2,1011020,1,Alabama,AL,Shelby County,Columbiana,Columbiana city,City,place,35051,205,44835274,261034,33.191452,-86.615618,54606,31930,57348,926.031000


In [5]:
income_dat = income_dat[~income_dat["State_Name"].isin(["Alaska", "Hawaii", "Puerto Rico"])]
income_dat = income_dat[income_dat["Mean"] > 0]
mean_US = income_dat["Mean"].describe()["mean"]
state_gcoder = geocode_states("US-48")
mean_income_state = income_dat.groupby("State_Name", as_index=False)["Mean"].mean()

In [8]:
import numpy as np
from scipy.interpolate import griddata

def interpolate_us(lons, lats, values, step, method):
    # method : "linear", "cubic" or "nearest".
    
    # target grid to interpolate to
    grid_lons = np.arange(-125, -66, step)
    grid_lats = np.arange(25, 52, step)
    grid_lons, grid_lats = np.meshgrid(grid_lons, grid_lats)
    
    # interpolate
    grid_values = griddata((lons, lats), values, (grid_lons, grid_lats), method)

    # lon, lat, value DataFrame
    return pd.DataFrame(dict(lon=grid_lons.flatten(), 
                           lat=grid_lats.flatten(), 
                           value=grid_values.flatten()))
X = income_dat.Lon
Y = income_dat.Lat
Z = income_dat.Mean
mean_income_interpolated = interpolate_us(X, Y, Z, step=.3, method="linear")

In [10]:
# A gradient color palette. 
# We will borrow HEX color codes from the Brewer's 'PiYG' palette:
#   - https://colorbrewer2.org/#type=diverging&scheme=PiYG&n=11
# The US income average is a good choice for the scales `midpoint`.
fill_PiYG= scale_fill_gradient2(name="Mean income", 
                                low="#8e0152",mid="#f7f7f7",high="#276419", 
                                midpoint=mean_US)
color_PiYG = scale_color_gradient2(name="Mean income", 
                                   low="#8e0152",mid="#f7f7f7",high="#276419", 
                                   midpoint=mean_US)

# Define some setting to use on plots later on:
# - Remove axis.
# - Define plot coordinate system and size.
map_settings = (theme(axis="blank", panel_grid="blank") +
                coord_fixed(1.27) +
                ggsize(785, 350))

# Customize the tooltip.
tooltip_scatter=(layer_tooltips()
    .format('Mean', '.2s')
    .line("Mean income|$@Mean"))

# Apply 'spatial union' operation.
states_union = state_gcoder.get_boundaries().geometry.unary_union

# Pick the biggest polygon.
states_union = max(states_union, key=lambda p: p.area)
boundary_us = gpd.GeoDataFrame({"geometry": [states_union]})

from shapely.geometry import Point

mask = mean_income_interpolated.apply(lambda p: states_union.contains(Point(p.lon, p.lat)), axis=1)
mean_income_interpolated_masked = mean_income_interpolated[mask]

# Customize the choropleth tooltip.
tooltip_tiles=(layer_tooltips()
    .format('value', '.2s')
    .line("Mean income|$@value"))

# Create a tile map with 'masked' data.
(ggplot(mean_income_interpolated_masked) + 
 geom_tile(aes("lon", "lat", fill="value"), tooltips=tooltip_tiles) + 
 geom_map(map=state_gcoder, size=.5, color="dark_magenta") +
 map_settings + fill_PiYG)

/var/folders/p7/1qllz46s60lfswf2l4vv4qtr0000gn/T/ipykernel_40881/1598375761.py:28: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  states_union = max(states_union, key=lambda p: p.area)
